### 실습 목차
* 1. RNN 실습
  * 1-1. RNN 구현
  * 1-2. RNN 학습
* 2. LSTM과 GRU 실습
  * 2-1. LSTM과 GRU 구현
  * 2-2. LSTM과 GRU 학습
* 3. One-to-one / One-to-many / Many-to-one / Many-to-many RNNs 구현 및 실습
  * 3-1. One-to-one RNN 실습
  * 3-2. One-to-Many RNN 실습
  * 3-3. Many-to-One RNN 실습
  * 3-4. Many-to-Many RNN 실습
* 4. Gradient Vanishing / Exploding

In [1]:
import torch
import numpy as np

from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class SimpleRNN(nn.Module): # SimpleRNN 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super().__init__() # nn.Module의 초기화 함수 상속
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.D = n_inputs # 입력 차원의 크기 지정
        self.K = n_outputs # 출력 차원의 크기 지정

        # RNN 모듈을 생성
        self.rnn = nn.RNN(input_size=self.D, # 입력 차원의 크기 지정
                          hidden_size=self.M, # 은닉 상태의 크기 지정
                          nonlinearity='tanh', # 활성화 함수로 tanh를 사용
                          batch_first=True) # 배치 차원이 먼저 오도록 설정
        
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 hidden state를 0으로 초기화

        # get RNN unit output
        out, _ = self.rnn(X, h0) # RNN에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

In [ ]:
model = SimpleRNN(n_inputs=2, n_hidden=20, n_outputs=2).to(device) # SimpleRNN 모델을 생성하고, GPU로 전송
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

# 더미 입력 데이터 예제
inputs = torch.from_numpy(np.array([[[1, 2], [3, 4], [5, 6]]], dtype=np.float32)).to(device)

for epoch in range(300): # 300번의 에폭 동안 학습을 진행
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(inputs) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, torch.tensor([1]).to(device))  # 더미 타겟 데이터로 손실(loss)을 계산
    loss.backward() # 역전파를 통해 기울기를 계산
    optimizer.step() # 최적화 알고리즘을 통해 파라미터를 업데이트

    if (epoch+1) % 10 == 0: # 10 에폭마다 손실을 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

In [ ]:
class LSTM(nn.Module): # LSTM 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super().__init__() # nn.Module의 초기화 함수 상속
        self.D = n_inputs # 입력 차원의 크기 지정
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.K = n_outputs # 출력 차원의 크기 지정

        # LSTM 모듈을 생성
        self.lstm = nn.LSTM(input_size=self.D, # 입력 차원의 크기 지정
                            hidden_size=self.M, # 은닉 상태의 크기 지정
                            batch_first=True) # 배치 차원이 먼저 오도록 설정
        
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 hidden state를 0으로 초기화
        c0 = torch.zeros(1, X.size(0), self.M).to(X.device) # LSTM의 초기 cell state를 0으로 초기화

        # LSTM에 입력을 전달하고 출력을 받음
        out, _ = self.lstm(X, (h0, c0)) ## LSTM은 hidden state와 cell state를 튜플로 반환함.

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 hidden state만 사용하여 선형 변환을 수행

        return out

In [ ]:
# LSTM 학습
model = LSTM(n_inputs=2, n_hidden=20, n_outputs=2).to(device) # LSTM 모델을 생성
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

for epoch in range(300): # 300회의 에포크동안 학습을 진행
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(inputs) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, torch.tensor([1]).to(device))  # 예시로 사용할 목표 텐서 생성
    loss.backward() # 역전파를 수행하여 기울기를 계산
    optimizer.step() # 최적화 알고리즘을 통해 파라미터 업데이트

    if (epoch+1) % 30 == 0: # 30 에포크마다 손실을 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

In [ ]:
class GRU(nn.Module): # GRU 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super().__init__() # nn.Module의 초기화 함수 상속
        self.D = n_inputs # 입력 차원의 크기 지정
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        
        # GRU 모듈을 생성
        self.gru = nn.GRU(input_size=self.D, # 입력 차원의 크기 지정
                          hidden_size=self.M, # 은닉 상태의 크기 지정
                          batch_first=True) # 배치 차원이 먼저 오도록 설정
        
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정

        # get RNN unit output
        out, _ = self.gru(X, h0) ## output, hidden state

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

In [ ]:
# GRU 학습
model = GRU(n_inputs=2, n_hidden=20, n_outputs=2).to(device) # GRU 모델 인스턴스 생성
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

for epoch in range(300):
    model.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, torch.tensor([1]).to(device))  # A dummy target example
    loss.backward()
    optimizer.step()

    if (epoch+1) % 30 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

In [ ]:
# One-to-One
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성, 1~4 사이의 정수 1000개를 랜덤하게 생성
Y = np.square(X) # 타겟 데이터 생성, 입력 데이터의 제곱을 타겟으로 설정

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).squeeze(-1).to(device) # 타겟 데이터를 텐서로 변환

model = SimpleRNN(n_inputs=1, n_hidden=40, n_outputs=1).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

for epoch in range(4000): # 4000회 반복하여 학습
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델에 입력 데이터 전달하여 예측값 계산
    loss = criterion(outputs, Y) # 예측값과 타겟값을 이용하여 손실 계산
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 업데이트

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print(f"Input: 2.0, Output: {model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

In [2]:
#### 입력의 배수 10개를 타겟으로 설정
# 아래의 One-to-Many는 하나의 숫자 입력 데이터를 받고, 입력된 데이터의 배수 10개를 예측하는 RNN 모델을 구현합니다.

# One-to-Many
class SimpleRNNOne2Many(nn.Module): # One-to-Many 모델 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(SimpleRNNOne2Many, self).__init__()
        self.D = n_inputs
        self.M = n_hidden
        self.K = n_outputs
        self.rnn = nn.RNN(
            input_size=self.D,
            hidden_size=self.M,
            nonlinearity='tanh',
            batch_first=True) # RNN 모듈 생성
        self.fc = nn.Linear(self.M, self.K) # 선형 변환 정의

    def forward(self, X): # 순전파 함수 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정
        out, _ = self.rnn(X, h0) # RNN에 입력을 전달하고 출력을 받음. [batch_size, seq_len, hidden_dim] = [1000, 1, 40]
        out = self.fc(out) # 출력에 선형 변환을 수행 [batch_size, seq_len, output_dim] = [1000, 1, 10]
        return out.view(-1, 10) # 출력을 적절한 형태로 변환 [1000, 10]

X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성, 1~4 사이의 정수 1000개를 랜덤하게 생성
Y = np.array([[i*j for i in range(1, 11)] for j in X.squeeze()]) # 타겟 데이터 생성, 입력의 배수 10개를 타겟으로 설정
print(X.shape, Y.shape)
print(X[0], Y[0])

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 타겟 데이터를 텐서로 변환

model = SimpleRNNOne2Many(n_inputs=1, n_hidden=40, n_outputs=10).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

for epoch in range(4000): # 4000회 반복하여 학습
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델에 입력 데이터 전달하여 예측값 계산
    loss = criterion(outputs.squeeze(), Y) # 예측값과 타겟값을 이용하여 손실 계산
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 업데이트

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: 2.0")
output = [round(num, 1) for num in model(X_test).squeeze().tolist()] # 테스트 데이터에 대한 예측값 계산
answer = list(range(2, 21, 2)) # 정답 리스트 생성

for o, a in zip(output, answer): # 예측값과 정답을 비교하여 출력
    print(f"Output: {o}, 정답: {a}")

(1000, 1, 1) (1000, 10)
[[2]] [ 2  4  6  8 10 12 14 16 18 20]


In [ ]:
# 아래의 Many-to-One은 여러 개의 숫자 입력 데이터를 받고, 입력된 데이터의 총 합을 예측하는 RNN 모델을 구현합니다.
# Many-to-One
class ManyToOneRNN(nn.Module): # Many-to-One 모델 클래스 선언
    def __init__(self, input_size, hidden_size, output_size):
        super(ManyToOneRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 모듈 생성
        self.fc = nn.Linear(hidden_size, output_size) # 선형 변환 정의

    def forward(self, x): # 순전파 함수 정의
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # 초기 은닉 상태를 0으로 설정
        out, _ = self.rnn(x, h0) # RNN에 입력을 전달하고 출력을 받음
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

X = np.random.randint(1, 15, size=(50000, 6, 1)) # 입력 데이터 생성, 1~14 사이의 정수 50000개를 랜덤하게 생성
Y = np.array([np.sum(x) for x in X]) # 타겟 데이터 생성, 모든 입력 데이터의 총합이 타겟

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 타겟 데이터를 텐서로 변환

model = ManyToOneRNN(input_size=1, hidden_size=50, output_size=1).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

for epoch in range(4000):
    model.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.squeeze(), Y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0], [4.0], [6.0], [8.0], [10.0], [11.0]]], dtype=torch.float32).to(device)
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {X_test.squeeze().tolist()}")
output = round(model(X_test).item(), 1)
answer = round(sum(X_test.squeeze().tolist()), 1)

print(f"Output: {output}, 정답: {answer}")

In [ ]:
#### 입력 데이터 리스트의 누적 합 리스트가 정답
# 아래의 Many-to-Many 구조는 여러 개의 숫자 입력 데이터를 받고, 입력된 데이터의 누적합을 예측하는 RNN 모델을 구현합니다.
# 예를 들어, 1,3,5를 입력으로 받으면 RNN 모델은 이들의 누적합인 1,4,9를 출력하도록 학습됩니다.

# Many-to-Many
class ManyToManyRNN(nn.Module): # ManyToManyRNN 클래스 선언
    def __init__(self, input_size, hidden_size, output_size):
        super(ManyToManyRNN, self).__init__() # nn.Module의 초기화 함수 상속
        self.hidden_size = hidden_size # 은닉 상태(hidden state)의 크기를 지정
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 모듈을 생성
        self.fc = nn.Linear(hidden_size, output_size) # 출력을 위한 선형 변환을 정의

    def forward(self, x): # 순전파 함수를 정의
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # 초기 은닉 상태를 0으로 설정
        out, _ = self.rnn(x, h0) # RNN에 입력을 전달하고 출력을 받음
        out = self.fc(out) # 모든 시간 단계의 출력에 대해 선형 변환을 수행

        return out

# 데이터 생성
# 각각의 리스트는 0~30 사이의 랜덤한 정수를 가지는 5개의 정수로 구성, 이러한 리스트를 3000개 생성
X = np.array([[[np.random.randint(0, 31)] for _ in range(5)] for _ in range(3000)])
Y = np.array([np.cumsum(x) for x in X]) # 정답은 각 리스트의 누적합

X = torch.from_numpy(X.astype(np.float32)) # numpy 배열을 PyTorch 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)) # numpy 배열을 PyTorch 텐서로 변환

model = ManyToManyRNN(1, 60, 1) # 모델 생성

criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

# 학습
for epoch in range(4000): # 4000번의 에폭 동안 학습
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(X) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, Y.view_as(outputs)) # 손실 계산
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 갱신

    if (epoch+1) % 100 == 0: # 100 에폭마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# 추론
X_test = torch.tensor([[[i+10] for i in range(5)]], dtype=torch.float32) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {list(range(10, 15))}")
output = [round(num, 1) for num in model(X_test).squeeze().tolist()] # 모델의 출력 계산
answer = list(np.cumsum(range(10, 15))) # 정답 계산

for o, a in zip(output, answer): # 모델의 출력과 정답 비교
    print(f"Output: {o}, 정답: {a}")